## 자연어 임베딩

### 임베딩층 - 내부적으로는 연산을 해서 단어와 단어 사이의 관계를 계산해서 밀집벡터를 만든다.

In [3]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import os, pathlib, shutil, random

#데이터 다운로드 
def download():
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"

    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)

    print("Download complete!")


#압축풀기 : 프로그램 호출 -> 프로세스 , tar 라이브러리가 있어야 한다 
def release():
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True) #tar 프로그램 가동하기 
    #tar.gz => linux에서는 파일을 여러개를 한번에 압축을 못함 tar라는 형식으로 압축할 모든 파일을 하나로 묶어서 패키지로 만든다음에 
    #          압축을 한다.  tar , gz가동  그래서 압축풀고 다시 패키지도 풀어야 한다. 
    #          tar  -xvzf 파일명   형태임         
    print("압축풀기 완료")


#train => train과 validation으로 나눠야 한다. , train 폴더에 있는 unsup 폴더는 직접 지워냐 한다. 
#라벨이 2개 여야 한다. 

#라벨링 
def labeling(): 
    base_dir = pathlib.Path("aclImdb") 
    val_dir = base_dir/"val"   # pathlib 객체에  / "디렉토리" => 결과가 문자열이 아니다 
    train_dir = base_dir/"train"

    for category in ("neg", "pos"):
        os.makedirs(val_dir/category)  #디렉토리를 만들고 
        files = os.listdir(train_dir/category) #해당 카테고리의 파일 목록을 모두 가져온다 
        random.Random(1337).shuffle(files) #파일을 랜덤하게 섞어서 복사하려고 파일 목록을 모두 섞는다 
        num_val_samples = int(0.2 * len(files)) 
        val_files = files[-num_val_samples:] #20%만 val폴더로 이동한다 
        for fname in val_files:
            shutil.move(train_dir/category/fname, val_dir/category/fname )    

download() #파일 다운받기 = 용량이 너무 커서 8192만큼씩 잘라서 저장하는 코드임 
release()
labeling()

Download complete!
압축풀기 완료


In [4]:
#데이터셋을 활용해서 디렉토리로부터 파일을 불러와서 벡터화를 진행한다 
import keras 
batch_size = 32 #한번에 읽어올 양 
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", #디렉토리명 
    batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", #디렉토리명 
    batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", #디렉토리명 
    batch_size=batch_size
)
#데이터셋은 알아서  inputs, targets 을 반복해서 갖고 온다. 우리한테 필요한거는 inputs만이다
for inputs, targets in train_ds: #실제 읽어오는 데이터 확인 
    print("inputs.shape", inputs.shape)
    print("inputs.dtype", inputs.dtype)
    print("targets.shape", targets.shape)
    print("targets.dtype", targets.dtype)
    print("inputs[0]", inputs[:3])
    print("targets[0]", targets[:3])
    break #하나만 출력해보자 
#0이 부정 1이 긍정 -> 폴더명을 정렬해서 0,1,2 이런식으로 라벨링을 한다 neg -0 pos-1 

Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
inputs.shape (32,)
inputs.dtype <dtype: 'string'>
targets.shape (32,)
targets.dtype <dtype: 'int32'>
inputs[0] tf.Tensor(
[b'I stumbled on this little gem a few months ago on IFC, and just watched it again today. If there was an Academy Award category for "Best Modernistic Recreation of Classic Hollywood Styles" this film would have won hands down. Gibson Frazier is amazing, channeling the spirit of Buster Keaton and Harold Lloyd with a script that reads like a Robert Heinlein short story. (Or perhaps Harlan Ellison...) Hang your belief at the door and flow with the fantasy. You are not going to learn any deep truths here. Except perhaps how to make a really snappy film on small change. The plot (what there is of it, and be fair, not many reporter/gangster film from the 20\'s and 30\'s did either) moves Frazier from the clean black and white world of New York to 

In [5]:
#시퀀스를 만들어야 한다 
max_length = 600  #한 평론에서 사용하는 단어는 최대 길이를 600개라고 보자  
max_tokens = 20000 #자주 사용하는 단어 20000 개만 쓰겠다 

text_vectorization = TextVectorization( 
    max_tokens = max_tokens,
    output_mode = "int", #임베딩 층을 사요하려면 반드시 int여야 한다
    output_sequence_length = max_length  
)

text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds) #어휘사전을 만들어야 한다 

int_train_ds = train_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=1 )
int_val_ds = val_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=1 )
int_test_ds = test_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=1 )

#내부구조 살짝 보기 
for item in int_train_ds:
    print(item)
    break 

(<tf.Tensor: shape=(32, 600), dtype=int64, numpy=
array([[ 1164,    10,     7, ...,     0,     0,     0],
       [   10,   216,     5, ...,     0,     0,     0],
       [  593, 13747,  2108, ...,     0,     0,     0],
       ...,
       [   11,   197,     1, ...,     0,     0,     0],
       [   15,  1398,     8, ...,     0,     0,     0],
       [   38,    73,    69, ...,     0,     0,     0]])>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)>)


In [ ]:
# 임베딩층 - 내부적으로는 연산을 해서 단어와 단어 사이의 관계를 계산해서 밀집벡터를 만든다.
# 원핫인코딩 - 메모리를 너무 많이 차지함 최대한 한문장을 표현하는데 만일 최대 20000 단어까지 처리한다면
# 한문장당 20000개가 필요, 희소행렬 요소가 거의 다 0인데 그 중 몇개가 값이 있을 때, 학습 시 속도가 엄청 느리다.
# 임베딩층을 사용한다. => 단어와 단어사이의 거리를 재는 방식인데, 비슷비슷한 단어가 같은 문장에 나타난다고 전제하고 문제를 해결한다.
# 틀릴 수 있다.
# 케라스가 Embedding 레이어를 제공한다. 이 레이어는 반드시 정수 인덱스를 받아야 한다. 시퀀스를 받아서 밀집벡터를 만든다.
# 알고리즘 공개 안함. 파이썬은 소스 공개 안할 방법이 없다.  ProgramData/anaconda3/envs/가상환경/libs/site-packages

from keras import models, layers
import tensorflow as tf 
embedding_dim = 600 
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs) # 미리 학습된 임베딩층으로 바꿀 수 있다.
# 입력벡터크기는 20000, 출력벡터는 256(특별한 의미 없이 마음대로)의 크기를 갖는다.
print(embedded.shape)
#양방향 RNN 을 가동시킴 
x = layers.Bidirectional( layers.LSTM(32))(embedded) 
x = layers.Dropout(0.5)(x) 
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs) 
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)
print("테스트셋 ", model.evaluate(int_test_ds))

(None, None, 256)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        73,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
